In [18]:
#sqldf() provides a convenient interface of running SQL statement on data frames. 
#Similarly, Python also offers multiple ways to interact between SQL and Pandas DataFrames by 
#leveraging the lightweight SQLite engine.

#import the library SQLite
import sqlite3

SQLite in general, is a server-less database that can be used within almost all programming languages including Python. Server-less means there is no need to install a separate server to work with SQLite so you can connect directly with the database. 

SQLite is a lightweight database that can provide a relational database management system with zero-configuration because there is no need to configure or setup anything to use it.

In [133]:
# To use SQLite3 in Python, Create a connection object which will connect us to the database and will let us execute the SQL statements.
conn = sqlite3.connect('demo2.db')

# To execute SQLite statements in Python, you need a cursor object. You can create it using the cursor() method.
cur = conn.cursor()

# Now we can use the cursor object to call the execute() method to execute any SQL queries.

In [137]:
#design a table in the sqlite db
sql = """
    create table sensor (
        sid         integer primary key not null,
        name        text,
        notes       text
    );"""
cur.execute(sql)

OperationalError: table sensor already exists

In [136]:
print(sql)



    create table sensor (
        sid         integer primary key not null,
        name        text,
        notes       text
    );


[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

In [138]:
sql = "insert into sensor(sid, name, notes) values (%d, '%s', '%s');"
#sql = "insert into sensor(sid, name, notes) values (121, "B"", "Acadgild");"

In [139]:
print(sql)

insert into sensor(sid, name, notes) values (%d, '%s', '%s');


In [140]:
for (uid, name, notes) in [(201, 'Alpha', 'Sensor for weight'), \
                           (202, 'Beta', 'Sensor for conductivity'),
                           (203, 'Gamma', 'Sensor for surface oxides'),
                           (204, 'Delta', 'Sensor for length'),
                           (205, 'Epsilon', 'Sensor for x-ray'),
                           (206, 'Zeta', 'Color checker 9000'),
                           (207, 'Eta', 'Ultra-violet detector'), ]:
    cur.execute(sql % (uid, name, notes))


In [141]:
print(sql)

insert into sensor(sid, name, notes) values (%d, '%s', '%s');


In [142]:
cur.execute("select * from sensor")
rows = cur.fetchall()
# for row in rows:
#     print(row)
print(rows)

[(201, 'Alpha', 'Sensor for weight'), (202, 'Beta', 'Sensor for conductivity'), (203, 'Gamma', 'Sensor for surface oxides'), (204, 'Delta', 'Sensor for length'), (205, 'Epsilon', 'Sensor for x-ray'), (206, 'Zeta', 'Color checker 9000'), (207, 'Eta', 'Ultra-violet detector')]


In [144]:
cur.execute("update sensor set name = 'Mohit' where sid = 204")
cur.execute("select * from sensor")
rows = cur.fetchall()
for row in rows:
    print(row)

(201, 'Alpha', 'Sensor for weight')
(202, 'Beta', 'Sensor for conductivity')
(203, 'Gamma', 'Sensor for surface oxides')
(204, 'Mohit', 'Sensor for length')
(205, 'Epsilon', 'Sensor for x-ray')
(206, 'Zeta', 'Color checker 9000')
(207, 'Eta', 'Ultra-violet detector')


In [145]:
cur.execute("delete from sensor where sid = 207")
cur.execute("select * from sensor")
rows = cur.fetchall()
for row in rows:
    print(row)

(201, 'Alpha', 'Sensor for weight')
(202, 'Beta', 'Sensor for conductivity')
(203, 'Gamma', 'Sensor for surface oxides')
(204, 'Mohit', 'Sensor for length')
(205, 'Epsilon', 'Sensor for x-ray')
(206, 'Zeta', 'Color checker 9000')


In [85]:
import sqlalchemy

In [86]:
sqlalchemy.__version__

'1.2.15'

SQLAlchemy provides a nice “Pythonic” way of interacting with databases.The SQLAlchemy Object Relational Mapper presents a method of associating user-defined Python classes with database tables, and instances of those classes (objects) with rows in their corresponding tables. 

In [2]:
from sqlalchemy import create_engine

In [3]:
# To start interacting with the database we first we need to establish a connection.
engine = create_engine('sqlite:///:memory:', echo=True)

Some examples of connecting to various databases - https://docs.sqlalchemy.org/en/latest/core/engines.html#postgresql

In [4]:
#The echo flag is a shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard logging module. 

In [5]:
from sqlalchemy.ext.declarative import declarative_base

In [6]:
Base = declarative_base()

In [7]:
from sqlalchemy import Column, Integer, String

In [8]:
class User(Base):
...     __tablename__ = 'users'
...
...     id = Column(Integer, primary_key=True)
...     name = Column(String)
...     fullname = Column(String)
...     password = Column(String)
...
...     def __repr__(self):
...        return "<User(name='%s', fullname='%s', password='%s')>" % (
...                             self.name, self.fullname, self.password)

In [9]:
#With our User class constructed via the Declarative system, we have defined information about our table,
#known as table metadata. The object used by SQLAlchemy to represent this information for a specific table 
#is called the Table object, 
#and here Declarative has made one for us.

In [10]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

#The MetaData is a registry which includes the ability to emit a limited set of schema generation commands to the database. As our SQLite database does not actually have a users table present, we can use MetaData to issue CREATE TABLE statements to the database for all tables that don’t yet exist. 

In [12]:
Base.metadata.create_all(engine)

2019-11-30 16:06:11,407 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-11-30 16:06:11,409 INFO sqlalchemy.engine.base.Engine ()


In [13]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [14]:
ed_user.name

'ed'

In [15]:
ed_user.password

'edspassword'

In [16]:
str(ed_user.id)

'None'

In [17]:
from sqlalchemy.orm import sessionmaker

In [18]:
Session = sessionmaker(bind=engine)

In [103]:
Session = sessionmaker()

In [104]:
#when you create your engine with create_engine(), connect it to the Session using configure()

In [105]:
Session.configure(bind=engine) 

In [106]:
session = Session()

In [107]:
#Adding and Updating Objects

In [108]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [109]:
session.add(ed_user)

In [110]:
our_user = session.query(User).filter_by(name='ed').first() 

2019-03-24 08:32:21,193 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-24 08:32:21,196 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-03-24 08:32:21,197 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')
2019-03-24 08:32:21,199 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-03-24 08:32:21,200 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [111]:
our_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

In [68]:
#We can add more User objects at once using add_all():

In [112]:
session.add_all([
...     User(name='wendy', fullname='Wendy Williams', password='foobar'),
...     User(name='mary', fullname='Mary Contrary', password='xxg527'),
...     User(name='fred', fullname='Fred Flinstone', password='blah')])

In [113]:
#the password for Ed isn’t too secure, so lets change it:

In [114]:
ed_user.password = 'f8s7ccs'

#We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via commit(). 

In [116]:
session.commit()

2019-03-24 08:35:06,977 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2019-03-24 08:35:06,979 INFO sqlalchemy.engine.base.Engine ('f8s7ccs', 1)
2019-03-24 08:35:06,980 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-03-24 08:35:06,981 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2019-03-24 08:35:06,983 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-03-24 08:35:06,984 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2019-03-24 08:35:06,985 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-03-24 08:35:06,986 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')
2019-03-24 08:35:06,988 INFO sqlalchemy.engine.base.Engine COMMIT


In [117]:
#commit() flushes the remaining changes to the database, and commits the transaction. 

In [118]:
ed_user.name = 'Edwardo'

In [119]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')

In [120]:
session.add(fake_user)

In [121]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2019-03-24 08:36:26,075 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-24 08:36:26,077 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2019-03-24 08:36:26,078 INFO sqlalchemy.engine.base.Engine (1,)
2019-03-24 08:36:26,080 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2019-03-24 08:36:26,081 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2019-03-24 08:36:26,082 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-03-24 08:36:26,083 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2019-03-24 08:36:26,086 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2019-03-24 08:36:26,087 INFO sqlalchemy.engine.base.Engine ('Ed

[<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

In [122]:
session.rollback()

2019-03-24 08:36:26,701 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [123]:
ed_user.name

2019-03-24 08:36:27,583 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-24 08:36:27,586 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2019-03-24 08:36:27,587 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [124]:
fake_user in session

False

In [125]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2019-03-24 08:36:29,050 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2019-03-24 08:36:29,052 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

#A Query object is created using the query() method on Session. This function takes a variable number of arguments, which can be any combination of classes and class-instrumented descriptors.

In [126]:
for instance in session.query(User).order_by(User.id):
...     print(instance.name, instance.fullname)

2019-03-24 08:36:37,204 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2019-03-24 08:36:37,206 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [127]:
for name, fullname in session.query(User.name, User.fullname):
...     print(name, fullname)

2019-03-24 08:37:05,363 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-03-24 08:37:05,365 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [172]:
#Here’s a rundown of some of the most common operators used in filter():

In [128]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)

In [129]:
query.all() #all() returns a list:

2019-03-24 08:37:50,519 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-03-24 08:37:50,521 INFO sqlalchemy.engine.base.Engine ('%ed',)


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fred', fullname='Fred Flinstone', password='blah')>]

In [130]:
query.first()

2019-03-24 08:38:09,719 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2019-03-24 08:38:09,721 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>